### Formula 1 Data Exploration

### Formula 1 Data Exploration

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import sys
import datetime
import seaborn as sns

#starting in 2006 because new qualyfing standards start that year

### Lap Times Times Data

In [2]:
#Import Lap Times Data
#2010 races = 337 - 355
#fix nans when combining laptimes for meanlaptime



In [3]:
#laptimes_raw1 and laptimes_raw2 have the same data. I am slicing laptime
laptimes_raw = pd.read_csv('lapTimes.csv')

In [4]:
#take all laps before end of 2010 season
laptimes2010 = laptimes_raw[laptimes_raw['raceId']<=355]

In [5]:
#cut off all laps before start of 2010 season
laptimes2010 = laptimes2010[laptimes2010['raceId']>=337]

In [6]:
#add laptimes for 841+
laptimes2011_2017 = laptimes_raw[laptimes_raw['raceId']>840]

In [7]:
laptimes2011_2017.describe()

,raceId,driverId,lap,position,milliseconds
count,157826.000000,157826.000000,157826.000000,157826.000000,1.578260e+05
mean,910.751327,433.854986,29.850633,10.327430,1.017314e+05
std,44.195158,397.061838,18.058670,5.865332,1.141521e+05
min,841.000000,1.000000,1.000000,1.000000,6.741100e+04
25%,871.000000,13.000000,14.000000,5.000000,8.714725e+04
50%,905.000000,807.000000,29.000000,10.000000,9.801000e+04
75%,952.000000,821.000000,44.000000,15.000000,1.066808e+05
max,988.000000,843.000000,78.000000,24.000000,7.507547e+06


In [8]:
#2010 season ran from races 337-355
laptimes2010.describe()

,raceId,driverId,lap,position,milliseconds
count,22534.000000,22534.000000,22534.000000,22534.000000,2.253400e+04
mean,346.247537,161.758276,29.207464,10.702893,1.031021e+05
std,5.343727,294.184685,17.859642,6.092753,9.533963e+04
min,337.000000,1.000000,1.000000,1.000000,7.385100e+04
25%,342.000000,10.000000,14.000000,5.000000,8.552500e+04
50%,346.000000,20.000000,28.000000,10.000000,9.608000e+04
75%,351.000000,67.000000,43.000000,16.000000,1.106142e+05
max,355.000000,812.000000,78.000000,24.000000,2.942501e+06


In [9]:
laptimes2006_2009 = laptimes_raw[laptimes_raw['raceId'] <= 70]

In [10]:
laptimes2006_2017 = pd.concat([laptimes2010, laptimes2006_2009, laptimes2011_2017])

In [11]:
#2006-2010 lap times
laptimes2006_2017.describe()

,raceId,driverId,lap,position,milliseconds
count,256478.000000,256478.000000,256478.000000,256478.000000,2.564780e+05
mean,601.617589,285.926056,29.905415,10.126779,9.925210e+04
std,400.902813,375.706461,18.160026,5.764546,9.504869e+04
min,1.000000,1.000000,1.000000,1.000000,6.741100e+04
25%,59.000000,10.000000,14.000000,5.000000,8.400800e+04
50%,864.000000,21.000000,29.000000,10.000000,9.505400e+04
75%,926.000000,814.000000,44.000000,15.000000,1.053920e+05
max,988.000000,843.000000,78.000000,24.000000,7.507547e+06


In [12]:
laptimes2006_2017.head()

,raceId,driverId,lap,position,time,milliseconds
360067,337,18,1,8,2:10.721,130721
360068,337,18,2,8,2:04.853,124853
360069,337,18,3,8,2:04.279,124279
360070,337,18,4,8,2:04.154,124154
360071,337,18,5,8,2:03.955,123955


In [13]:
#create series of mean lap time in milliseconds
laptimes_mean_laps = laptimes2006_2017.groupby(['raceId', 'driverId'], as_index = False)['milliseconds'].mean()


In [14]:
#create new column for mean lap time data named 'meanlaptime'
laptimes_mean_laps['meanlaptime'] = laptimes_mean_laps['milliseconds']

In [15]:
#delete 'milliseconds' with redundant meanlaptime data
del laptimes_mean_laps['milliseconds']

In [16]:
#merge meanlaptime back onto laptimes2006_2017
laptimes2006_2017 = pd.merge(laptimes2006_2017, laptimes_mean_laps, on = ['driverId','raceId'], how = 'left')

In [17]:
#create a new column for fast lap time

In [18]:
laptimes_fast_laps = laptimes2006_2017.groupby(['raceId', 'driverId'], as_index = False)['milliseconds'].min()

In [19]:
#create new column for fastest lap time data named 'fastlaptime'
laptimes_fast_laps['fastlaptime'] = laptimes_fast_laps['milliseconds']

In [20]:
#delete 'milliseconds' with redundant fast lap time data
del laptimes_fast_laps['milliseconds']

In [21]:
#merge fast lap times back onto laptimes2006_2017
laptimes2006_2017 = pd.merge(laptimes2006_2017, laptimes_fast_laps, on = ['driverId','raceId'], how = 'left')

In [22]:
laptimes2006_2017.isna().sum()

raceId          0
driverId        0
lap             0
position        0
time            0
milliseconds    0
meanlaptime     0
fastlaptime     0
dtype: int64

In [23]:
laptimes = laptimes2006_2017.drop_duplicates(subset=['raceId', 'driverId'], keep = 'first')

In [24]:
del laptimes['time']

In [25]:
laptimes.describe()

,raceId,driverId,lap,position,milliseconds,meanlaptime,fastlaptime
count,4754.000000,4754.000000,4754.000000,4754.000000,4.754000e+03,4754.000000,4754.000000
mean,598.779344,288.246950,1.586243,11.105385,1.156812e+05,101768.061935,93371.375473
std,401.131286,376.082264,5.172943,6.215977,7.927013e+04,23077.370179,13135.950815
min,1.000000,1.000000,1.000000,1.000000,7.150600e+04,69134.126761,67411.000000
25%,59.250000,10.000000,1.000000,6.000000,9.837775e+04,88246.728745,83047.750000
50%,863.000000,21.000000,1.000000,11.000000,1.094835e+05,99113.079545,93241.000000
75%,918.000000,814.000000,1.000000,16.000000,1.225400e+05,108663.059091,101977.250000
max,988.000000,843.000000,67.000000,24.000000,3.702636e+06,507859.222222,297225.000000


### Pit Stop Data

# DID NOT USE, data is not available pre 2011

In [26]:

#import pitstop data
pitstops = pd.read_csv('pitStops.csv')

In [27]:
#Only data for race 841+
pitstops.raceId.min()

841

In [28]:
pitstops.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [8]:
#convert time to int object in hundreths of a second
def time_to_hundreths(string):
    split_string = string.split(':')
    mins_as_hund =  int(split_string[0])*60*1000
    seconds_as_hund = int(split_string[1])*1000
    hundredths = int(split_string[2])*10
    return mins_as_hund + seconds_as_hund + hundredths
   


In [9]:
pitstops['pit_time'] = pitstops['time'].apply(time_to_hundreths)

In [10]:
pitstops.head(10)

,raceId,driverId,stop,lap,time,duration,milliseconds,pit_time
0,841,153,1,1,17:05:23,26.898,26898,1025230
1,841,30,1,1,17:05:52,25.021,25021,1025520
2,841,17,1,11,17:20:48,23.426,23426,1040480
3,841,4,1,12,17:22:34,23.251,23251,1042340
4,841,13,1,13,17:24:10,23.842,23842,1044100
5,841,22,1,13,17:24:29,23.643,23643,1044290
6,841,20,1,14,17:25:17,22.603,22603,1045170
7,841,814,1,14,17:26:03,24.863,24863,1046030
8,841,816,1,14,17:26:50,25.259,25259,1046500
9,841,67,1,15,17:27:34,25.342,25342,1047340


In [51]:
#pitstops['pit_time'] = pitstops['time'] - pitstops['time_next'][1]

In [11]:
#find mean time elapsed when drivers take their first pit
pit1_mean_time = pitstops[pitstops['stop'] == 1].groupby(['raceId'], as_index = False)['pit_time'].mean()

In [12]:
pit1_mean_time = pit1_mean_time.rename(columns = {'pit_time':'pit1_mean_time'})
pit1_mean_time.head()

,raceId,pit1_mean_time
0,841,1.045824e+06
1,842,9.873027e+05
2,843,9.293533e+05
3,844,9.201848e+05
4,845,8.618004e+05


In [13]:
#find mean time elapsed when drivers take their second pit
pit2_mean_time = pitstops[pitstops['stop'] == 2].groupby(['raceId'], as_index = False)['pit_time'].mean()

In [14]:
pit2_mean_time = pit2_mean_time.rename(columns = {'pit_time':'pit2_mean_time'})
pit2_mean_time.head()

,raceId,pit2_mean_time
0,841,1.070904e+06
1,842,1.011992e+06
2,843,9.562091e+05
3,844,9.405883e+05
4,845,8.845387e+05


In [15]:
pit_stops = pd.merge(pit1_mean_time, pitstops, on = ['raceId'], how = 'left')

In [16]:
pit_stops_means = pd.merge(pit2_mean_time, pit_stops, on = ['raceId'], how = 'left')

In [17]:
pit_stops_means['stops>2'] = pit_stops_means['stop'][pit_stops_means['stop']>2]

In [19]:
pit_stops_means['stops>2'].isna().sum()

4921

In [20]:
#if this column is not null then the driver pitted more than twice in the race
pit_stops_means['stops>2'] = pit_stops_means['stops>2'].notna()
pit_stops_means.head()

,raceId,pit2_mean_time,pit1_mean_time,driverId,stop,lap,time,duration,milliseconds,pit_time,stops>2
0,841,1.070904e+06,1.045824e+06,153,1,1,17:05:23,26.898,26898,1025230,False
1,841,1.070904e+06,1.045824e+06,30,1,1,17:05:52,25.021,25021,1025520,False
2,841,1.070904e+06,1.045824e+06,17,1,11,17:20:48,23.426,23426,1040480,False
3,841,1.070904e+06,1.045824e+06,4,1,12,17:22:34,23.251,23251,1042340,False
4,841,1.070904e+06,1.045824e+06,13,1,13,17:24:10,23.842,23842,1044100,False


In [21]:
# Create column for pit timing deviations from race mean timing for pit stop 1 or pit stop 2
def pit_deviation(row):
    if row['stop'] == 1:
        return row['pit1_mean_time'] - row['pit_time']
    elif row['stop'] == 2:
        return row['pit2_mean_time'] - row['pit_time']
    else:
        return np.nan

In [22]:
#apply function to create pit deviation column
pit_stops_means['pit_dev'] = pit_stops_means.apply(pit_deviation, axis = 1)

In [23]:
#shift pit_dev from 2nd pit stop into new column to prepare to consolidate to driver+race level from pit level
pit_stops_means['pit_2_dev'] = pit_stops_means.groupby(['raceId', 'driverId'])['pit_dev'].shift(-1)
pit_stops_means.head()

,raceId,pit2_mean_time,pit1_mean_time,driverId,stop,lap,time,duration,milliseconds,pit_time,stops>2,pit_dev,pit_2_dev
0,841,1.070904e+06,1.045824e+06,153,1,1,17:05:23,26.898,26898,1025230,False,20594.285714,19844.117647
1,841,1.070904e+06,1.045824e+06,30,1,1,17:05:52,25.021,25021,1025520,False,20304.285714,18824.117647
2,841,1.070904e+06,1.045824e+06,17,1,11,17:20:48,23.426,23426,1040480,False,5344.285714,6614.117647
3,841,1.070904e+06,1.045824e+06,4,1,12,17:22:34,23.251,23251,1042340,False,3484.285714,4864.117647
4,841,1.070904e+06,1.045824e+06,13,1,13,17:24:10,23.842,23842,1044100,False,1724.285714,-1375.882353


In [24]:
pit_stops_means.describe()

,raceId,pit2_mean_time,pit1_mean_time,driverId,stop,lap,milliseconds,pit_time,pit_dev,pit_2_dev
count,6251.000000,6.251000e+03,6.251000e+03,6251.000000,6251.000000,6251.000000,6.251000e+03,6.251000e+03,4.921000e+03,2147.000000
mean,907.860342,9.534763e+05,9.256292e+05,429.468245,1.836346,24.813470,4.774453e+04,9.464803e+05,-1.182842e-13,85.014023
std,44.327698,1.055181e+05,1.050769e+05,397.079565,0.935314,14.150022,1.791970e+05,1.094411e+05,1.329130e+04,14171.345729
min,841.000000,8.166110e+05,7.958714e+05,1.000000,1.000000,1.000000,1.289700e+04,7.843100e+05,-1.142822e+05,-76800.909091
25%,868.000000,8.908615e+05,8.625285e+05,13.000000,1.000000,13.000000,2.176800e+04,8.735000e+05,-5.375294e+03,-5562.954545
50%,902.000000,9.027946e+05,8.737145e+05,807.000000,2.000000,24.000000,2.334000e+04,9.084200e+05,4.000000e+01,-91.666667
75%,950.000000,9.645259e+05,9.364311e+05,821.000000,2.000000,35.000000,2.554700e+04,9.761250e+05,4.733000e+03,5045.586124
max,988.000000,1.269659e+06,1.229606e+06,843.000000,6.000000,74.000000,2.011266e+06,1.313080e+06,1.174091e+05,117409.090909


In [25]:
#converting 'stop' column to total pit stop in race column
total_race_pits = pd.DataFrame(pit_stops_means.groupby(['raceId','driverId'], as_index = True)['stop'].max())
total_race_pits.head()

stop
raceId driverId      
841    1            2
       2            2
       3            1
       4            3
       5            1

In [26]:
#merging 'total_race_pits' to the rest of put stop data
pit_stop = pd.merge(total_race_pits, pit_stops_means, on = ['raceId','driverId'], how = 'left')
pit_stop.describe()

,raceId,driverId,stop_x,pit2_mean_time,pit1_mean_time,stop_y,lap,milliseconds,pit_time,pit_dev,pit_2_dev
count,6251.000000,6251.000000,6251.000000,6.251000e+03,6.251000e+03,6251.000000,6251.000000,6.251000e+03,6.251000e+03,4921.000000,2147.000000
mean,907.860342,429.468245,2.672372,9.534763e+05,9.256292e+05,1.836346,24.813470,4.774453e+04,9.464803e+05,0.000000,85.014023
std,44.327698,397.079565,1.043457,1.055181e+05,1.050769e+05,0.935314,14.150022,1.791970e+05,1.094411e+05,13291.304393,14171.345729
min,841.000000,1.000000,1.000000,8.166110e+05,7.958714e+05,1.000000,1.000000,1.289700e+04,7.843100e+05,-114282.173913,-76800.909091
25%,868.000000,13.000000,2.000000,8.908615e+05,8.625285e+05,1.000000,13.000000,2.176800e+04,8.735000e+05,-5375.294118,-5562.954545
50%,902.000000,807.000000,3.000000,9.027946e+05,8.737145e+05,2.000000,24.000000,2.334000e+04,9.084200e+05,40.000000,-91.666667
75%,950.000000,821.000000,3.000000,9.645259e+05,9.364311e+05,2.000000,35.000000,2.554700e+04,9.761250e+05,4733.000000,5045.586124
max,988.000000,843.000000,6.000000,1.269659e+06,1.229606e+06,6.000000,74.000000,2.011266e+06,1.313080e+06,117409.090909,117409.090909


In [27]:
#rename stop_x 'pit_stops'
pit_stop = pit_stop.rename(columns = {'stop_x': 'pit_stops'})

In [28]:
#check pit_2_dev column for a driver to make sure the transformation was correct
pit_stops_means[pit_stops_means['driverId'] == 1]

,raceId,pit2_mean_time,pit1_mean_time,driverId,stop,lap,time,duration,milliseconds,pit_time,stops>2,pit_dev,pit_2_dev
11,841,1.070904e+06,1.045824e+06,1,1,16,17:28:24,23.227,23227,1048240,False,-2415.714286,-8385.882353
33,841,1.070904e+06,1.045824e+06,1,2,36,17:59:29,23.199,23199,1079290,False,-8385.882353,NaN
49,842,1.011992e+06,9.873027e+05,1,1,12,16:24:40,22.552,22552,984400,False,2902.727273,6512.000000
72,842,1.011992e+06,9.873027e+05,1,2,24,16:45:48,22.611,22611,1005480,False,6512.000000,NaN
89,842,1.011992e+06,9.873027e+05,1,3,37,17:08:28,25.382,25382,1028280,True,NaN,NaN
103,842,1.011992e+06,9.873027e+05,1,4,52,17:34:48,22.466,22466,1054480,True,NaN,NaN
116,843,9.562091e+05,9.293533e+05,1,1,15,15:29:49,20.878,20878,929490,False,-136.666667,9009.090909
132,843,9.562091e+05,9.293533e+05,1,2,25,15:47:20,20.533,20533,947200,False,9009.090909,NaN
150,843,9.562091e+05,9.293533e+05,1,3,38,16:09:38,20.567,20567,969380,True,NaN,NaN
166,844,9.405883e+05,9.201848e+05,1,1,9,15:17:38,20.833,20833,917380,False,2804.782609,5498.260870


In [29]:
#1/5 pit stops were > 2nd stop
pit_stop.isna().sum()  

raceId               0
driverId             0
pit_stops            0
pit2_mean_time       0
pit1_mean_time       0
stop_y               0
lap                  0
time                 0
duration             0
milliseconds         0
pit_time             0
stops>2              0
pit_dev           1330
pit_2_dev         4104
dtype: int64

In [33]:

#pit_stop = pit_stop.drop_duplicates(subset=['raceId', 'driverId'], keep = 'first')

In [ ]:
pit_stops_means_summed = pit_stop

In [34]:
#pit_stops_means_summed = pit_stop.groupby(['raceId', 'driverId'], as_index = False)['pit_dev','pit_2_dev'].sum(index = 0)
pit_stops_means_summed['pit_dev_combine'] = pit_stop['pit_dev'] + pit_stop['pit_2_dev']

In [35]:
pit_stops_means_summed.describe()

,raceId,driverId,pit_stops,pit2_mean_time,pit1_mean_time,stop_y,lap,milliseconds,pit_time,pit_dev,pit_2_dev,pit_dev_combine
count,2790.000000,2790.000000,2790.000000,2.790000e+03,2.790000e+03,2790.000000,2790.000000,2.790000e+03,2.790000e+03,2774.000000,2104.000000,2090.000000
mean,910.579928,436.515054,2.240860,9.493683e+05,9.215691e+05,1.038351,15.389247,4.169096e+04,9.227109e+05,-65.798525,-7.082959,1256.522627
std,44.184616,396.824269,0.983576,1.002598e+05,9.952917e+04,0.223163,9.328268,1.523111e+05,1.028514e+05,12570.160922,14238.989137,23785.037742
min,841.000000,1.000000,1.000000,8.166110e+05,7.958714e+05,1.000000,1.000000,1.416000e+04,7.843100e+05,-114282.173913,-76800.909091,-189153.083004
25%,871.000000,15.000000,2.000000,8.915233e+05,8.645405e+05,1.000000,9.000000,2.208325e+04,8.623000e+05,-5164.772257,-5643.615561,-7881.250000
50%,905.000000,807.000000,2.000000,9.027946e+05,8.759705e+05,1.000000,14.000000,2.367300e+04,8.797900e+05,106.862745,-174.442724,638.311688
75%,952.000000,821.000000,3.000000,9.641850e+05,9.293533e+05,1.000000,21.000000,2.584400e+04,9.463175e+05,4589.895833,4962.205882,9896.032895
max,988.000000,843.000000,6.000000,1.269659e+06,1.229606e+06,4.000000,56.000000,2.010361e+06,1.292140e+06,64617.826087,117409.090909,182026.916996


In [36]:
#join laptimes and pit data

pits_laps = pd.merge(laptimes, pit_stop, on = ['driverId','raceId'], how = 'left')

# pits_laps = pits_laps.rename(columns = {'time_x':'time_at_pit', 'milliseconds_x':'pit_duration', 'milliseconds_y':'lap_time_mil', 'time_y':'lap_time'})

NameError: name 'laptimes' is not defined

In [878]:
pits_laps = pits_laps.rename(columns = {'milliseconds_x':'median_race_lap','milliseconds_y':'pit_duration'})

In [883]:
pits_laps.head()

,raceId,driverId,median_race_lap,pit_stops,pit2_mean_time,pit1_mean_time,stop_y,lap,time,duration,pit_duration,pit_time,stops>2,pit_dev
0,841,1,91818.0,2.0,1.070904e+06,1.045824e+06,1.0,16.0,17:28:24,23.227,23227.0,1048240.0,False,-2415.714286
1,841,2,94694.0,2.0,1.070904e+06,1.045824e+06,1.0,15.0,17:27:41,22.994,22994.0,1047410.0,False,-1585.714286
2,841,3,94657.5,1.0,1.070904e+06,1.045824e+06,1.0,16.0,17:29:00,23.716,23716.0,1049000.0,False,-3175.714286
3,841,4,91868.0,3.0,1.070904e+06,1.045824e+06,1.0,12.0,17:22:34,23.251,23251.0,1042340.0,False,3484.285714
4,841,5,96391.0,1.0,1.070904e+06,1.045824e+06,1.0,17.0,17:31:11,24.865,24865.0,1051110.0,False,-5285.714286


### Qualifying Times Data

In [29]:
# add qualifying times

qualifying_raw = pd.read_csv('qualifying.csv')

In [32]:
qualifying2011_2017 = pd.DataFrame(qualifying_raw[qualifying_raw['raceId'] > 840])

In [33]:
qualifying2011_2017.shape

(2980, 9)

In [34]:
#qualifying2011_2017.head()

In [35]:
#take all races before end of 2010 season
qualifying2010 = qualifying_raw[qualifying_raw['raceId']<=355]

In [36]:
#cut off all races before start of 2010 season
qualifying2010 = qualifying2010[qualifying_raw['raceId']>=337]

/Users/christianbuerkle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [37]:
qualifying2010.shape

(456, 9)

In [38]:
#all races from 2006 - 2010 season
qualifying2006_2010 = qualifying_raw[qualifying_raw['raceId']<=70]

In [39]:
#qualifying2006_2010.describe()

In [40]:
qualifying2006_2017 = pd.concat([qualifying2006_2010, qualifying2010,qualifying2011_2017])

In [41]:
#qualifying2006_2017.describe()

In [42]:
qualifying2006_2017['q_position'] = qualifying2006_2017['position']

In [48]:
# del qualifying2006_2017 ['position']

In [49]:
# del qualifying2006_2017['number']

In [50]:
# del qualifying2006_2017['qualifyId']

In [51]:
# del qualifying2006_2017['constructorId']

In [52]:
qualifying2006_2017.describe()

,raceId,driverId,q_position
count,4914.000000,4914.000000,4914.000000
mean,595.531950,287.364265,11.429589
std,401.997493,375.502588,6.369710
min,1.000000,1.000000,1.000000
25%,59.000000,10.000000,6.000000
50%,862.000000,21.000000,11.000000
75%,924.000000,814.000000,17.000000
max,988.000000,843.000000,24.000000


In [53]:
qualifying = qualifying2006_2017

In [54]:
#no duplcates in this data set
qualifying = qualifying.drop_duplicates(subset=['raceId', 'driverId'], keep = 'first')

In [55]:
qualifying.shape

(4914, 6)

In [56]:
laps_qual = pd.merge( qualifying, laptimes, on = ['driverId','raceId'], how = 'left')

In [57]:
laps_qual.isna().sum()

raceId             0
driverId           0
q1                39
q2              1369
q3              2736
q_position         0
lap              179
position         179
milliseconds     179
meanlaptime      179
fastlaptime      179
dtype: int64

###  Results Data

In [58]:
#Importing results data

In [59]:
results_raw = pd.read_csv('results.csv')

In [60]:
results_raw.shape

(23777, 18)

In [61]:
results2011_2017 = pd.DataFrame(results_raw[results_raw['raceId'] > 840])

In [62]:
results2011_2017.shape

(3001, 18)

In [63]:
results2010 = results_raw[results_raw['raceId']<=355]

In [64]:
#cut off all races before start of 2010 season
results2010 = results2010[results_raw['raceId']>=337]

/Users/christianbuerkle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [65]:
results2010.shape

(456, 18)

In [66]:
results2006_2010 = results_raw[results_raw['raceId']<=70]

In [67]:
results2006_2010.describe()

,resultId,raceId,driverId,constructorId,number,grid,position,positionOrder,points,laps,milliseconds,fastestLap,rank,statusId
count,1478.000000,1478.000000,1478.000000,1478.000000,1478.000000,1478.000000,1121.000000,1478.000000,1478.000000,1478.000000,7.000000e+02,1400.000000,1400.000000,1478.000000
mean,2326.076455,36.242219,16.755074,6.316644,11.679973,11.066982,8.687779,11.080514,1.833897,51.509472,5.657218e+06,37.609286,10.560000,6.841001
std,3363.166421,20.178138,18.197542,4.193025,6.784407,6.132832,4.891750,6.130460,2.930933,20.235836,6.287680e+05,18.063411,5.866331,8.735085
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,4.252092e+06,2.000000,1.000000,1.000000
25%,370.250000,19.000000,7.000000,3.000000,6.000000,6.000000,5.000000,6.000000,0.000000,46.000000,5.289318e+06,21.000000,5.750000,1.000000
50%,739.500000,37.000000,15.000000,6.000000,11.000000,11.000000,9.000000,11.000000,0.000000,57.000000,5.559964e+06,39.000000,10.500000,3.000000
75%,1108.750000,54.000000,21.000000,9.000000,17.000000,16.000000,13.000000,16.000000,3.000000,66.000000,5.849038e+06,52.000000,15.250000,11.000000
max,16086.000000,70.000000,155.000000,23.000000,23.000000,22.000000,21.000000,22.000000,10.000000,78.000000,7.668773e+06,77.000000,22.000000,73.000000


In [68]:
results2006_2017 = pd.concat([results2006_2010, results2010,results2011_2017])

In [71]:
results2006_2017.shape


(4935, 18)

In [72]:
laps_qual_results = pd.merge(results2006_2017, laps_qual, on = ['driverId','raceId'], how = 'left')

In [73]:
laps_qual_results.shape

(4935, 27)

In [75]:
laps_qual_results.to_csv('project2_laps_qual_results.csv')

In [84]:
# del laps_qual_results['fastestLapTime']

# del laps_qual_results['positionText']

# del laps_qual_results['constructorId']

# del laps_qual_results['resultId']

# del laps_qual_results['number']

# # del laps_qual_results['position']

# del laps_qual_results['laps']

# del laps_qual_results['time']

# del laps_qual_results['milliseconds_x']

# del laps_qual_results['rank']

# del laps_qual_results['statusId']

# del laps_qual_results['q1']

# del laps_qual_results['q2']

# del laps_qual_results['q3']

# del laps_qual_results['milliseconds_y']

In [86]:
laps_qual_results.to_csv('laps_qual_result_trim.csv')

In [87]:
laps_qual_results.shape

(4935, 14)